Connect pymongo and create DB/collection

In [5]:
# start pymongo and create db/collection
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
dbDev = client["Dev"]
col = dbDev["stock_data"]

Get monthly time series stock data, put in dict

In [23]:
from bson.objectid  import ObjectId  
from config import api_keys
import json
import ssl
import urllib.request

# select highest protocol supported by client / server
context = ssl.SSLContext(ssl.PROTOCOL_TLS)

ticker = "GOOG"
stock_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=' + ticker + '&apikey=' + api_keys["stocks"]

with urllib.request.urlopen(stock_url, context=context) as url:  # decode JSON data
    #data = url.read().decode()  # load to string
    data = json.loads(url.read().decode())  # load straight to dict

type(data)


dict

In [24]:
data


{'Meta Data': {'1. Information': 'Monthly Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'GOOG',
  '3. Last Refreshed': '2019-04-24 12:31:16',
  '4. Time Zone': 'US/Eastern'},
 'Monthly Time Series': {'2019-04-24': {'1. open': '1184.1000',
   '2. high': '1269.0000',
   '3. low': '1182.0000',
   '4. close': '1261.7926',
   '5. volume': '16107450'},
  '2019-03-29': {'1. open': '1124.9000',
   '2. high': '1231.7900',
   '3. low': '1123.3000',
   '4. close': '1173.3100',
   '5. volume': '31658936'},
  '2019-02-28': {'1. open': '1112.4000',
   '2. high': '1147.0000',
   '3. low': '1086.0000',
   '4. close': '1119.9200',
   '5. volume': '28934646'},
  '2019-01-31': {'1. open': '1016.5700',
   '2. high': '1117.3300',
   '3. low': '1014.0700',
   '4. close': '1116.3700',
   '5. volume': '30714651'},
  '2018-12-31': {'1. open': '1123.1400',
   '2. high': '1124.6500',
   '3. low': '970.1100',
   '4. close': '1035.6100',
   '5. volume': '40256461'},
  '2018-11-30': {'1. open': '1075

Will need to restructure/cleanse data (can't have periods in mongodb)

In [19]:
data["Meta Data"]["2. Symbol"]

'GOOG'

In [29]:
timeseries = data["Monthly Time Series"]
timeseries

{'2019-04-24': {'1. open': '1184.1000',
  '2. high': '1269.0000',
  '3. low': '1182.0000',
  '4. close': '1261.7926',
  '5. volume': '16107450'},
 '2019-03-29': {'1. open': '1124.9000',
  '2. high': '1231.7900',
  '3. low': '1123.3000',
  '4. close': '1173.3100',
  '5. volume': '31658936'},
 '2019-02-28': {'1. open': '1112.4000',
  '2. high': '1147.0000',
  '3. low': '1086.0000',
  '4. close': '1119.9200',
  '5. volume': '28934646'},
 '2019-01-31': {'1. open': '1016.5700',
  '2. high': '1117.3300',
  '3. low': '1014.0700',
  '4. close': '1116.3700',
  '5. volume': '30714651'},
 '2018-12-31': {'1. open': '1123.1400',
  '2. high': '1124.6500',
  '3. low': '970.1100',
  '4. close': '1035.6100',
  '5. volume': '40256461'},
 '2018-11-30': {'1. open': '1075.8000',
  '2. high': '1095.5700',
  '3. low': '996.0200',
  '4. close': '1094.4300',
  '5. volume': '36735570'},
 '2018-10-31': {'1. open': '1199.8900',
  '2. high': '1209.9600',
  '3. low': '995.8300',
  '4. close': '1076.7700',
  '5. vol

In [32]:
for k,v in timeseries.items():
    print(v["1. open"])

1184.1000
1124.9000
1112.4000
1016.5700
1123.1400
1075.8000
1199.8900
1204.2700
1228.0000
1099.0000
1099.3500
1013.6600
1022.8200
1107.8700
1162.6100
1048.3400
1015.8000
1017.2100
959.9800
941.1300
932.3800
912.1800
968.9500
901.9400
829.2200
828.8500
799.6800
778.8100
757.4400
782.8900
774.2500
769.2500
761.0900
692.2000
734.5300
697.6300
738.6000
703.6200
750.4600
743.0000
747.1100
711.0600
608.3700
602.3600
625.3400
524.7300
536.7900
538.4300
548.6000
560.5300
531.7300
529.0100
538.9000
555.5000
576.0100
571.8500
570.4000
578.3200
560.7000
527.1100
558.7100
